In [1]:
!pwd

/Users/shihosato/src/github.com/twinkle13531/master_degree/202012/Summary


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys, os
sys.path.append(os.path.join('..', 'Random'))
sys.path.append(os.path.join('..', 'SA'))
sys.path.append(os.path.join('..', 'QA'))
sys.path.append(os.path.join('..', '4ti2', 'functions'))

In [4]:
import QA_DwaveSampler_AutoEmbedding as qda
import random_exact_test_functions as ret
import SA_exact_test_functions as net
import zsolve_t1 as zt
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import statistics
import math

In [5]:
time_dic = {}

In [6]:
y_dic = {'total_y_num':{}, 'mean_occurrence':{}, 'valid_y_num':{}}

In [7]:
per_time_dic = {}

In [8]:
mean_broken_chain_proportion = {}#only QA

In [9]:
p_dic = {}

In [10]:
chain_strength_dic = {20:10, 25:15, 30:20, 35:20, 40:30}

In [114]:
bit_list = [20, 25, 30, 35, 40]
num_list = [0, 1, 2, 3, 4]

## QA
計算時間の制約があるので一気に、まとめて計算する。


前回からの変更点
- calculation_time測定箇所がおかしかった→修正
- n, num_reads.たしかにnを増やすとvalid_y_numが増えるが、per_timeもcalculation_timeも*n弱ほど増える
  - どう頑張ってもvalid_y_numでSAには勝てない
  - n, num_reads = 100, 100となるとどうしても計算が時間かかりすぎる25bitでさえも
  - →n=1, num_reads=10000のtotal_timeで勝負する
  - 時間の平均、標準偏差などはtotal_timeにする（すべての手法）


In [12]:
n, num_reads = 1, 10000

In [13]:
time_dic['QA'] = {}
y_dic['total_y_num']['QA'] = {}
y_dic['mean_occurrence']['QA'] = {}
y_dic['valid_y_num']['QA'] = {}
per_time_dic['QA'] = {}
p_dic['QA'] = {}

In [14]:
for bit in [20, 25, 30, 35, 40]:
    time_dic['QA'][bit] = {}
    y_dic['total_y_num']['QA'][bit] = {}
    y_dic['mean_occurrence']['QA'][bit] = {}
    y_dic['valid_y_num']['QA'][bit] = {}
    per_time_dic['QA'][bit] = {}
    mean_broken_chain_proportion[bit] = {}
    p_dic['QA'][bit] = {}
    chain_strength = chain_strength_dic[bit]
    
    for num in [0, 1, 2, 3, 4]:
        path = '../input/ost{}_{}.csv'.format(bit, num)
        df = pd.read_csv(path, sep=',')
        bqm = qda.make_Hamiltonian(df)
        
        qqda = qda.QA_DSampler_AEmbedding(df, bqm, n, num_reads, chain_strength)
        valid_y_info_dic, calculation_time = qqda.find_valid_y_time_ntimes()
        
        if len(valid_y_info_dic) > 0:
            occurrence_list = [valid_y_info[0] for valid_y_info in list(valid_y_info_dic.values())]
            broken_list = [valid_y_info[1] for valid_y_info in list(valid_y_info_dic.values())]

            time_dic['QA'][bit][num] = calculation_time
            y_dic['total_y_num']['QA'][bit][num] = sum(occurrence_list)
            y_dic['mean_occurrence']['QA'][bit][num] = statistics.mean(occurrence_list)
            y_dic['valid_y_num']['QA'][bit][num] = len(valid_y_info_dic)
            per_time_dic['QA'][bit][num] = calculation_time / sum(occurrence_list)
            mean_broken_chain_proportion[bit][num] = statistics.mean(broken_list)
            p_dic['QA'][bit][num] = qqda.p_value(valid_y_info_dic)

        print('{}_{} finish'.format(bit, num))

20_0 finish
20_1 finish
20_2 finish
20_3 finish
20_4 finish
25_0 finish
25_1 finish
25_2 finish
25_3 finish
25_4 finish
30_0 finish
30_1 finish
30_2 finish
30_3 finish
30_4 finish
35_0 finish
35_1 finish
35_2 finish
35_3 finish
35_4 finish
40_0 finish
40_1 finish
40_2 finish
40_3 finish
40_4 finish


In [15]:
time_dic['QA']

{20: {0: 4.8007480826568605,
  1: 5.29829094078064,
  2: 4.8907990184021,
  3: 4.973049851913452,
  4: 4.945617887039185},
 25: {0: 4.76728770401001,
  1: 4.107626649398804,
  2: 4.130954000015259,
  3: 4.125704976577759,
  4: 4.125749084014893},
 30: {0: 4.857393952865601,
  1: 4.35492608215332,
  2: 4.321266147155762,
  3: 4.382329198379517,
  4: 4.98394915725708},
 35: {0: 4.5598540034484865,
  1: 4.569085809249878,
  2: 4.78381225730896,
  3: 6.331811162490845,
  4: 5.85717913772583},
 40: {0: 6.264639112014771,
  1: 6.066893073577881,
  2: 6.663262816925049,
  4: 6.245596858520508}}

In [16]:
y_dic['total_y_num']['QA']

{20: {0: 388, 1: 204, 2: 342, 3: 285, 4: 47},
 25: {0: 11, 1: 180, 2: 46, 3: 20, 4: 81},
 30: {0: 13, 1: 5, 2: 84, 3: 6, 4: 82},
 35: {0: 1, 1: 23, 2: 70, 3: 8, 4: 5},
 40: {0: 8, 1: 2, 2: 1, 4: 1}}

In [17]:
y_dic['mean_occurrence']['QA']

{20: {0: 1, 1: 1, 2: 1, 3: 1, 4: 1},
 25: {0: 1, 1: 1, 2: 1, 3: 1, 4: 1},
 30: {0: 1, 1: 1, 2: 1, 3: 1, 4: 1},
 35: {0: 1, 1: 1, 2: 1, 3: 1, 4: 1},
 40: {0: 1, 1: 1, 2: 1, 4: 1}}

In [18]:
y_dic['valid_y_num']['QA']

{20: {0: 261, 1: 157, 2: 200, 3: 247, 4: 41},
 25: {0: 11, 1: 175, 2: 44, 3: 20, 4: 81},
 30: {0: 13, 1: 5, 2: 83, 3: 6, 4: 82},
 35: {0: 1, 1: 23, 2: 70, 3: 8, 4: 5},
 40: {0: 8, 1: 2, 2: 1, 4: 1}}

In [19]:
per_time_dic['QA']

{20: {0: 0.012373062068703249,
  1: 0.025972014415591372,
  2: 0.014300581925152338,
  3: 0.017449297726012114,
  4: 0.10522591249019542},
 25: {0: 0.43338979127363725,
  1: 0.022820148052215578,
  2: 0.08980334782641868,
  3: 0.20628524882888793,
  4: 0.05093517387672707},
 30: {0: 0.3736456886819693,
  1: 0.8709852164306641,
  2: 0.05144364460899716,
  3: 0.7303881997299194,
  4: 0.060779867771427805},
 35: {0: 4.5598540034484865,
  1: 0.1986559047499947,
  2: 0.06834017510441372,
  3: 0.7914763953113556,
  4: 1.171435827545166},
 40: {0: 0.7830798890018463,
  1: 3.0334465367889405,
  2: 6.663262816925049,
  4: 6.245596858520508}}

In [20]:
p_dic['QA']

{20: {0: 0.09578544061302682,
  1: 0.20382165605095542,
  2: 0.35,
  3: 0.31983805668016196,
  4: 0.3902439024390244},
 25: {0: 0.09090909090909091,
  1: 0.10857142857142857,
  2: 0.09090909090909091,
  3: 0.15,
  4: 0.06172839506172839},
 30: {0: 0.0, 1: 0.4, 2: 0.060240963855421686, 3: 0.0, 4: 0.15853658536585366},
 35: {0: 0.0, 1: 0.13043478260869565, 2: 0.014285714285714285, 3: 0.0, 4: 0.0},
 40: {0: 0.125, 1: 0.0, 2: 0.0, 4: 0.0}}

## Random

前回からの変更点
- calculation_timeをret.find_valid_y(df, num_reads)の時間とした
```python
time_0 = time.time()
valid_y_dic = ret.find_valid_y(df, num_reads)
time_1 = time.time()
calculation_time = time_1 - time_0
```

In [21]:
num_reads = 10000

In [22]:
time_dic['random'] = {}
y_dic['total_y_num']['random'] = {}
y_dic['mean_occurrence']['random'] = {}
y_dic['valid_y_num']['random'] = {}
per_time_dic['random'] = {}
p_dic['random'] = {}

In [23]:
for bit in bit_list:
    time_dic['random'][bit] = {}
    y_dic['total_y_num']['random'][bit] = {}
    y_dic['mean_occurrence']['random'][bit] = {}
    y_dic['valid_y_num']['random'][bit] = {}
    per_time_dic['random'][bit] = {}
    p_dic['random'][bit] = {}
    
    for num in num_list:
        df_path = '../input/ost{}_{}.csv'.format(bit, num)
        df = pd.read_csv(df_path, sep=',')
        t_list = ret.calc_marginals(df)
        time_0 = time.time()
        valid_y_dic = ret.find_valid_y(df, num_reads)
        time_1 = time.time()
        calculation_time = time_1 - time_0
        
        if len(valid_y_dic) > 0:
            rm = ret.Random_method(df, valid_y_dic, num_reads, t_list)
            
            mean_occurrence_list = [i for i in list(valid_y_dic.values())]
            
            time_dic['random'][bit][num] = calculation_time
            y_dic['total_y_num']['random'][bit][num] = sum(mean_occurrence_list)
            y_dic['mean_occurrence']['random'][bit][num] = statistics.mean(mean_occurrence_list)
            y_dic['valid_y_num']['random'][bit][num] = len(valid_y_dic)
            per_time_dic['random'][bit][num] = calculation_time / len(valid_y_dic)
            p_dic['random'][bit][num] = rm.p_value()
        print('{}_{} finish'.format(bit, num))

20_0 finish
20_1 finish
20_2 finish
20_3 finish
20_4 finish
25_0 finish
25_1 finish
25_2 finish
25_3 finish
25_4 finish
30_0 finish
30_1 finish
30_2 finish
30_3 finish
30_4 finish
35_0 finish
35_1 finish
35_2 finish
35_3 finish
35_4 finish
40_0 finish
40_1 finish
40_2 finish
40_3 finish
40_4 finish


In [24]:
time_dic['random']

{20: {0: 0.22255897521972656,
  1: 0.17073702812194824,
  2: 0.05727791786193848,
  3: 0.19472885131835938,
  4: 0.18351507186889648},
 25: {0: 0.07721900939941406, 1: 0.11200618743896484, 4: 0.07980966567993164},
 30: {2: 0.14279603958129883, 3: 0.08059096336364746, 4: 0.06939125061035156},
 35: {1: 0.10467791557312012, 2: 0.15607309341430664},
 40: {}}

In [25]:
y_dic['total_y_num']['random']

{20: {0: 161, 1: 10, 2: 13, 3: 30, 4: 2},
 25: {0: 3, 1: 26, 4: 8},
 30: {2: 2, 3: 2, 4: 5},
 35: {1: 4, 2: 7},
 40: {}}

In [26]:
y_dic['mean_occurrence']['random']

{20: {0: 1.00625, 1: 1, 2: 1, 3: 1, 4: 1},
 25: {0: 1, 1: 1, 4: 1},
 30: {2: 1, 3: 1, 4: 1},
 35: {1: 1, 2: 1},
 40: {}}

In [27]:
y_dic['valid_y_num']['random']

{20: {0: 160, 1: 10, 2: 13, 3: 30, 4: 2},
 25: {0: 3, 1: 26, 4: 8},
 30: {2: 2, 3: 2, 4: 5},
 35: {1: 4, 2: 7},
 40: {}}

In [28]:
per_time_dic['random']

{20: {0: 0.001390993595123291,
  1: 0.017073702812194825,
  2: 0.004405993681687575,
  3: 0.006490961710611979,
  4: 0.09175753593444824},
 25: {0: 0.025739669799804688,
  1: 0.004307930286114032,
  4: 0.009976208209991455},
 30: {2: 0.07139801979064941, 3: 0.04029548168182373, 4: 0.013878250122070312},
 35: {1: 0.02616947889328003, 2: 0.022296156202043806},
 40: {}}

In [29]:
p_dic['random']

{20: {0: 0.2375,
  1: 0.1,
  2: 0.38461538461538464,
  3: 0.23333333333333334,
  4: 0.5},
 25: {0: 0.0, 1: 0.038461538461538464, 4: 0.375},
 30: {2: 0.0, 3: 0.0, 4: 0.0},
 35: {1: 0.25, 2: 0.0},
 40: {}}

## SA
- occurrence_list = list(valid_y_info_dic.values())箇所を変更

In [115]:
num_reads = 10000

In [116]:
time_dic['SA'] = {}
y_dic['total_y_num']['SA'] = {}
y_dic['mean_occurrence']['SA'] = {}
y_dic['valid_y_num']['SA'] = {}
per_time_dic['SA'] = {}
p_dic['SA'] = {}

In [117]:
for bit in bit_list:
    time_dic['SA'][bit] = {}
    y_dic['total_y_num']['SA'][bit] = {}
    y_dic['mean_occurrence']['SA'][bit] = {}
    y_dic['valid_y_num']['SA'][bit] = {}
    per_time_dic['SA'][bit] = {}
    p_dic['SA'][bit] = {}
    
    for num in num_list:
        
        path = '../input/ost{}_{}.csv'.format(bit, num)
        df = pd.read_csv(path, sep=',')
        
        time_0 = time.time()
        res = net.make_res_data(df, num_reads)
        valid_y_info_dic = net.find_valid_y(res)
        time_1 = time.time()
        calculation_time = time_1 - time_0

        occurrence_list = list(valid_y_info_dic.values())
        
        if len(valid_y_info_dic) > 0:
            rvd = net.SA_res_valid_dic(df, res, valid_y_info_dic, num_reads)
            
            time_dic['SA'][bit][num] = calculation_time
            y_dic['total_y_num']['SA'][bit][num] = sum(occurrence_list)
            y_dic['mean_occurrence']['SA'][bit][num] = statistics.mean(occurrence_list)
            y_dic['valid_y_num']['SA'][bit][num] = len(valid_y_info_dic)
            per_time_dic['SA'][bit][num] = calculation_time / len(valid_y_info_dic)
            p_dic['SA'][bit][num] = rvd.calc_p_value()
            
        print('{}_{} finish'.format(bit, num))

20_0 finish
20_1 finish
20_2 finish
20_3 finish
20_4 finish
25_0 finish
25_1 finish
25_2 finish
25_3 finish
25_4 finish
30_0 finish
30_1 finish
30_2 finish
30_3 finish
30_4 finish
35_0 finish
35_1 finish
35_2 finish
35_3 finish
35_4 finish
40_0 finish
40_1 finish
40_2 finish
40_3 finish
40_4 finish


In [118]:
time_dic['SA']

{20: {0: 9.18666696548462,
  1: 5.6547510623931885,
  2: 5.7256858348846436,
  3: 6.279776096343994,
  4: 5.75777792930603},
 25: {0: 10.265526056289673,
  1: 14.074274063110352,
  2: 10.197137117385864,
  3: 7.361194849014282,
  4: 11.226624011993408},
 30: {0: 15.163124084472656,
  1: 16.260918140411377,
  2: 15.577936172485352,
  3: 15.437527894973755,
  4: 15.91696572303772},
 35: {0: 18.32254910469055,
  1: 20.69321608543396,
  2: 19.49590492248535,
  3: 19.030593156814575,
  4: 18.11780095100403},
 40: {0: 21.85553789138794,
  1: 22.198148012161255,
  2: 22.350066900253296,
  3: 22.198980808258057,
  4: 21.001359939575195}}

In [119]:
y_dic['total_y_num']['SA']

{20: {0: 9756, 1: 9874, 2: 9840, 3: 9616, 4: 9639},
 25: {0: 9858, 1: 9790, 2: 9401, 3: 8557, 4: 9866},
 30: {0: 9747, 1: 9269, 2: 9935, 3: 9238, 4: 9894},
 35: {0: 9536, 1: 9804, 2: 9848, 3: 9601, 4: 9438},
 40: {0: 9661, 1: 9761, 2: 9647, 3: 9578, 4: 9578}}

In [120]:
y_dic['mean_occurrence']['SA']

{20: {0: 1, 1: 7, 2: 6, 3: 4, 4: 37},
 25: {0: 1, 1: 1, 2: 1, 3: 6, 4: 1},
 30: {0: 1, 1: 1, 2: 1, 3: 1, 4: 1},
 35: {0: 1, 1: 1, 2: 1, 3: 1, 4: 1},
 40: {0: 1, 1: 1, 2: 1, 3: 1, 4: 1}}

In [121]:
y_dic['valid_y_num']['SA']

{20: {0: 7247, 1: 1364, 2: 1630, 3: 2351, 4: 260},
 25: {0: 6809, 1: 9290, 2: 6853, 3: 1372, 4: 7822},
 30: {0: 8937, 1: 8590, 2: 9834, 3: 8901, 4: 9836},
 35: {0: 9499, 1: 9796, 2: 9848, 3: 9589, 4: 9430},
 40: {0: 9658, 1: 9759, 2: 9644, 3: 9577, 4: 9577}}

In [122]:
per_time_dic['SA']

{20: {0: 0.001267651023248878,
  1: 0.004145711922575651,
  2: 0.0035126906962482476,
  3: 0.0026711085054632044,
  4: 0.022145299728100115},
 25: {0: 0.0015076407778366386,
  1: 0.0015149918259537516,
  2: 0.0014879814850993528,
  3: 0.005365302368086211,
  4: 0.001435262594220584},
 30: {0: 0.0016966682426398855,
  1: 0.00189300560423881,
  2: 0.001584089502998307,
  3: 0.001734358824286457,
  4: 0.0016182356367464132},
 35: {0: 0.001928892420748558,
  1: 0.002112414871930784,
  2: 0.0019796816533799097,
  3: 0.0019846275061856894,
  4: 0.0019212938442209999},
 40: {0: 0.0022629465615435847,
  1: 0.002274633467789861,
  2: 0.002317510047724315,
  3: 0.002317947249478757,
  4: 0.002192895472441808}}

In [123]:
p_dic['SA']

{20: {0: 0.22602456188767767,
  1: 0.2155425219941349,
  2: 0.3325153374233129,
  3: 0.264568268821778,
  4: 0.4153846153846154},
 25: {0: 0.07783815538258188,
  1: 0.06167922497308934,
  2: 0.15409309791332262,
  3: 0.15743440233236153,
  4: 0.2527486576323191},
 30: {0: 0.0433031218529708,
  1: 0.3145518044237485,
  2: 0.20012202562538134,
  3: 0.10920121334681497,
  4: 0.06893045953639691},
 35: {0: 0.062427623960416886,
  1: 0.04797876684360964,
  2: 0.01726238830219334,
  3: 0.07268745437480446,
  4: 0.15461293743372218},
 40: {0: 0.039345620211223856,
  1: 0.06107183113023876,
  2: 0.03235172127747823,
  3: 0.06787094079565625,
  4: 0.026730708990289235}}

In [124]:
#path = '../input/ost{}_{}.csv'.format(bit, num)
#df = pd.read_csv(path, sep=',')

#num_reads = 10000

#res = net.make_res_data(df, num_reads)

In [125]:
#for i in res.data(['num_occurrences']):
#    print(i)

## たしか1ばかりだった

## exhaustive enumeration

In [41]:
enu_time_dic = {}
enu_time_dic[20] = [4.84, 0.02, 0.23, 0.38, 0.65]
enu_time_dic[25] = [5.47, 2894.31, 4.19, 0.23, 15.37]
enu_time_dic[30] = [120017.00, 105.15, 12792.04, 725.79]

In [42]:
#enu_time_dic[30] = [120017.00, 105.15, 12792.04, 725.79, ,]
#enu_time_dic[35] = [, , , , , ]
#enu_time_dic[40] = [64667.50, , , , ,]

In [43]:
time_dic['enumeration'] = {}
y_dic['valid_y_num']['enumeration'] = {}
per_time_dic['enumeration'] = {}
p_dic['enumeration'] = {}

In [44]:
for bit in [20, 25]:
    time_dic['enumeration'][bit] = {}
    y_dic['valid_y_num']['enumeration'][bit] = {}
    per_time_dic['enumeration'][bit] = {}
    p_dic['enumeration'][bit] = {}
    
    for num in [0, 1, 2, 3, 4]:
        
        path = '../input/ost{}_{}.csv'.format(bit, num)
        df = pd.read_csv(path, sep=',')
        
        zinhom_path = '../4ti2/{}bit/{}/{}bit_{}.zinhom'.format(bit, num, bit, num)
        zinhom_num, zinhom_list = zt.make_result_list(zinhom_path)
        
        if zinhom_num > 0:
            zrs = zt.zsolve_result_summerize(df, zinhom_list)
            
            time_dic['enumeration'][bit][num] = enu_time_dic[bit][num]
            y_dic['valid_y_num']['enumeration'][bit][num] = zinhom_num
            per_time_dic['enumeration'][bit][num] = enu_time_dic[bit][num] / zinhom_num
            p_dic['enumeration'][bit][num] = zrs.calucurate_p()

        print('{}_{} finish'.format(bit, num))

20_0 finish
20_1 finish
20_2 finish
20_3 finish
20_4 finish
25_0 finish
25_1 finish
25_2 finish
25_3 finish
25_4 finish


In [45]:
for bit in [30]:
    time_dic['enumeration'][bit] = {}
    y_dic['valid_y_num']['enumeration'][bit] = {}
    per_time_dic['enumeration'][bit] = {}
    p_dic['enumeration'][bit] = {}
    
    for num in [0, 1, 2, 3]:
        
        path = '../input/ost{}_{}.csv'.format(bit, num)
        df = pd.read_csv(path, sep=',')
        
        zinhom_path = '../4ti2/{}bit/{}/{}bit_{}.zinhom'.format(bit, num, bit, num)
        zinhom_num, zinhom_list = zt.make_result_list(zinhom_path)
        
        if zinhom_num > 0:
            zrs = zt.zsolve_result_summerize(df, zinhom_list)
            
            time_dic['enumeration'][bit][num] = enu_time_dic[bit][num]
            y_dic['valid_y_num']['enumeration'][bit][num] = zinhom_num
            per_time_dic['enumeration'][bit][num] = enu_time_dic[bit][num] / zinhom_num
            p_dic['enumeration'][bit][num] = zrs.calucurate_p()

        print('{}_{} finish'.format(bit, num))

30_0 finish
30_1 finish
30_2 finish
30_3 finish


In [46]:
time_dic['enumeration']

{20: {0: 4.84, 1: 0.02, 2: 0.23, 3: 0.38, 4: 0.65},
 25: {0: 5.47, 1: 2894.31, 2: 4.19, 3: 0.23, 4: 15.37},
 30: {0: 120017.0, 1: 105.15, 2: 12792.04, 3: 725.79}}

In [47]:
y_dic['valid_y_num']['enumeration']

{20: {0: 16464, 1: 1365, 2: 1638, 3: 2397, 4: 260},
 25: {0: 12600, 1: 89796, 2: 14700, 3: 1372, 4: 21000},
 30: {0: 58296, 1: 57159, 2: 429660, 3: 127440}}

In [48]:
per_time_dic['enumeration']

{20: {0: 0.00029397473275024293,
  1: 1.4652014652014653e-05,
  2: 0.00014041514041514042,
  3: 0.00015853149770546517,
  4: 0.0025},
 25: {0: 0.0004341269841269841,
  1: 0.03223205933449151,
  2: 0.0002850340136054422,
  3: 0.00016763848396501459,
  4: 0.0007319047619047618},
 30: {0: 2.058751886921916,
  1: 0.0018396053114995016,
  2: 0.029772471256342227,
  3: 0.00569515065913371}}

In [49]:
p_dic['enumeration']

{20: {0: 0.22959183673469388,
  1: 0.2153846153846154,
  2: 0.3333333333333333,
  3: 0.2640801001251564,
  4: 0.4153846153846154},
 25: {0: 0.08333333333333333,
  1: 0.0608044901777362,
  2: 0.15306122448979592,
  3: 0.15743440233236153,
  4: 0.248},
 30: {0: 0.04459997255386304,
  1: 0.31648559282002836,
  2: 0.20353302611367127,
  3: 0.11007532956685499}}

## mean, std dicについて
 - enumeration入(all_mean_time, all_std_time)と、なしを作る
   - 入はせっかくなので4サンプルずつ20, 25, 30bitまで
   - なしは5サンプルずつ20-40bitまで
 - 種類
   - time
   - per_time
   - valid_y_num
   - p_value//
   
   - total_y_num
   - mean_occurrence

In [126]:
method_list_all = ['enumeration', 'random', 'SA', 'QA']
method_list_35 = ['random', 'SA', 'QA']
method_list_40 = ['SA', 'QA']

bit_list_30 = [20, 25, 30]
bit_list_35 = [20, 25, 30, 35]
#bit_list_all = [20, 25, 30, 35, 40]

### time

In [127]:
mean_time_30 = {}
std_time_30 = {}

In [128]:
for method in all_method_list:
    mean_time_list = []
    std_time_list = []
    for bit in bit_list_30:
        time_list = list(time_dic[method][bit].values())[:4]
        mean_time_list.append(statistics.mean(time_list))
        std_time_list.append(statistics.pstdev(time_list))

    mean_time_30[method] = np.array(mean_time_list)
    std_time_30[method] = np.array(std_time_list)

In [129]:
mean_time_30

{'enumeration': array([1.3675000e+00, 7.2605000e+02, 3.3409995e+04]),
 'random': array([0.16132569, 0.08967829, 0.09759275]),
 'SA': array([ 6.71171999, 10.47453302, 15.60987657]),
 'QA': array([4.99072197, 4.28289333, 4.47897885])}

In [130]:
mean_time_35 = {}
std_time_35 = {}

In [131]:
for method in method_list_35:
    mean_time_list = []
    std_time_list = []
    for bit in bit_list_35:
        time_list = list(time_dic[method][bit].values())
        mean_time_list.append(statistics.mean(time_list))
        std_time_list.append(statistics.pstdev(time_list))

    mean_time_35[method] = np.array(mean_time_list)
    std_time_35[method] = np.array(std_time_list)

In [132]:
mean_time_35

{'random': array([0.16576357, 0.08967829, 0.09759275, 0.1303755 ]),
 'SA': array([ 6.52093158, 10.62495122, 15.6712944 , 19.13201284]),
 'QA': array([4.98170116, 4.25146448, 4.57997291, 5.22034847])}

In [133]:
mean_time_40 = {}
std_time_40 = {}

In [134]:
for method in method_list_40:
    mean_time_list = []
    std_time_list = []
    for bit in bit_list:
        time_list = list(time_dic[method][bit].values())
        mean_time_list.append(statistics.mean(time_list))
        std_time_list.append(statistics.pstdev(time_list))

    mean_time_40[method] = np.array(mean_time_list)
    std_time_40[method] = np.array(std_time_list)

In [135]:
mean_time_40

{'SA': array([ 6.52093158, 10.62495122, 15.6712944 , 19.13201284, 21.92081871]),
 'QA': array([4.98170116, 4.25146448, 4.57997291, 5.22034847, 6.31009797])}

### per_time

In [136]:
mean_per_time_30 = {}
std_per_time_30 = {}

for method in all_method_list:
    mean_time_list = []
    std_time_list = []
    for bit in bit_list_30:
        time_list = list(per_time_dic[method][bit].values())[:4]
        mean_time_list.append(statistics.mean(time_list))
        std_time_list.append(statistics.pstdev(time_list))

    mean_per_time_30[method] = np.array(mean_time_list)
    std_per_time_30[method] = np.array(std_time_list)

In [137]:
mean_per_time_30

{'enumeration': array([1.51893346e-04, 8.27971470e-03, 5.24014779e-01]),
 'random': array([0.00734041, 0.01334127, 0.04185725]),
 'SA': array([0.00289929, 0.00246898, 0.00172703]),
 'QA': array([0.01752374, 0.18807463, 0.50661569])}

In [138]:
mean_per_time_35 = {}
std_per_time_35 = {}

for method in method_list_35:
    mean_time_list = []
    std_time_list = []
    for bit in bit_list_35:
        time_list = list(per_time_dic[method][bit].values())
        mean_time_list.append(statistics.mean(time_list))
        std_time_list.append(statistics.pstdev(time_list))

    mean_per_time_35[method] = np.array(mean_time_list)
    std_per_time_35[method] = np.array(std_time_list)

In [139]:
mean_per_time_35

{'random': array([0.02422384, 0.01334127, 0.04185725, 0.02423282]),
 'SA': array([0.00674849, 0.00226224, 0.00170527, 0.00198538]),
 'QA': array([0.03506417, 0.16064674, 0.41744852, 1.35795246])}

In [140]:
mean_per_time_40 = {}
std_per_time_40 = {}

for method in method_list_40:
    mean_time_list = []
    std_time_list = []
    for bit in bit_list:
        time_list = list(per_time_dic[method][bit].values())
        mean_time_list.append(statistics.mean(time_list))
        std_time_list.append(statistics.pstdev(time_list))

    mean_per_time_40[method] = np.array(mean_time_list)
    std_per_time_40[method] = np.array(std_time_list)

In [141]:
mean_per_time_40

{'SA': array([0.00674849, 0.00226224, 0.00170527, 0.00198538, 0.00227319]),
 'QA': array([0.03506417, 0.16064674, 0.41744852, 1.35795246, 4.18134653])}

### valid_y_num

In [142]:
mean_valid_y_num_30 = {}
std_valid_y_num_30 = {}

for method in all_method_list:
    mean_num_list = []
    std_num_list = []
    for bit in bit_list_30:
        num_list = list(y_dic['valid_y_num'][method][bit].values())[:4]
        mean_num_list.append(statistics.mean(num_list))
        std_num_list.append(statistics.pstdev(num_list))

    mean_valid_y_num_30[method] = np.array(mean_num_list)
    std_valid_y_num_30[method] = np.array(std_num_list)

In [143]:
mean_valid_y_num_30

{'enumeration': array([  5466.  ,  29617.  , 168138.75]),
 'random': array([53.25      , 12.33333333,  3.        ]),
 'SA': array([3148. , 6081. , 9065.5]),
 'QA': array([216.25,  62.5 ,  26.75])}

In [144]:
mean_valid_y_num_35 = {}
std_valid_y_num_35 = {}

for method in method_list_35:
    mean_num_list = []
    std_num_list = []
    for bit in bit_list_35:
        num_list = list(y_dic['valid_y_num'][method][bit].values())
        mean_num_list.append(statistics.mean(num_list))
        std_num_list.append(statistics.pstdev(num_list))

    mean_valid_y_num_35[method] = np.array(mean_num_list)
    std_valid_y_num_35[method] = np.array(std_num_list)

In [145]:
mean_valid_y_num_35

{'random': array([43.        , 12.33333333,  3.        ,  5.5       ]),
 'SA': array([2570.4, 6429.2, 9219.6, 9632.4]),
 'QA': array([181.2,  66.2,  37.8,  21.4])}

In [146]:
mean_valid_y_num_40 = {}
std_valid_y_num_40 = {}

for method in method_list_40:
    mean_num_list = []
    std_num_list = []
    for bit in bit_list_40:
        num_list = list(y_dic['valid_y_num'][method][bit].values())
        mean_num_list.append(statistics.mean(num_list))
        std_num_list.append(statistics.pstdev(num_list))

    mean_valid_y_num_40[method] = np.array(mean_num_list)
    std_valid_y_num_40[method] = np.array(std_num_list)

In [147]:
mean_valid_y_num_40

{'SA': array([2570.4, 6429.2, 9219.6, 9643. ]),
 'QA': array([181.2,  66.2,  37.8,   3. ])}

### p_value

In [148]:
mean_p_30 = {}
std_p_30 = {}

for method in all_method_list:
    mean_p_list = []
    std_p_list = []
    for bit in bit_list_30:
        p_list = list(p_dic[method][bit].values())[:4]
        mean_p_list.append(statistics.mean(p_list))
        std_p_list.append(statistics.pstdev(p_list))

    mean_p_30[method] = np.array(mean_p_list)
    std_p_30[method] = np.array(std_p_list)

In [149]:
mean_p_30

{'enumeration': array([0.26059747, 0.11365836, 0.16867348]),
 'random': array([0.23886218, 0.13782051, 0.        ]),
 'SA': array([0.25966267, 0.11276122, 0.16679454]),
 'QA': array([0.24236129, 0.1100974 , 0.11506024])}

In [150]:
mean_p_35 = {}
std_p_35 = {}

for method in method_list_35:
    mean_p_list = []
    std_p_list = []
    for bit in bit_list_35:
        p_list = list(p_dic[method][bit].values())
        mean_p_list.append(statistics.mean(p_list))
        std_p_list.append(statistics.pstdev(p_list))

    mean_p_35[method] = np.array(mean_p_list)
    std_p_35[method] = np.array(std_p_list)

In [151]:
mean_p_35

{'random': array([0.29108974, 0.13782051, 0.        , 0.125     ]),
 'SA': array([0.29080706, 0.14075871, 0.14722172, 0.07099383]),
 'QA': array([0.27193781, 0.1004236 , 0.12375551, 0.0289441 ])}

In [152]:
mean_p_40 = {}
std_p_40 = {}

for method in method_list_40:
    mean_p_list = []
    std_p_list = []
    for bit in bit_list_40:
        p_list = list(p_dic[method][bit].values())
        mean_p_list.append(statistics.mean(p_list))
        std_p_list.append(statistics.pstdev(p_list))

    mean_p_40[method] = np.array(mean_p_list)
    std_p_40[method] = np.array(std_p_list)

In [153]:
mean_p_40

{'SA': array([0.29080706, 0.14075871, 0.14722172, 0.04547416]),
 'QA': array([0.27193781, 0.1004236 , 0.12375551, 0.03125   ])}

### total_y_num

In [154]:
mean_total_y_num_35 = {}
std_total_y_num_35 = {}

for method in method_list_35:
    mean_total_num_list = []
    std_total_num_list = []
    for bit in bit_list_35:
        total_num_list = list(y_dic['total_y_num'][method][bit].values())
        mean_total_num_list.append(statistics.mean(total_num_list))
        std_total_num_list.append(statistics.pstdev(total_num_list))

    mean_total_y_num_35[method] = np.array(mean_total_num_list)
    std_total_y_num_35[method] = np.array(std_total_num_list)

In [155]:
mean_total_y_num_35

{'random': array([43.2       , 12.33333333,  3.        ,  5.5       ]),
 'SA': array([9745, 9494, 9616, 9645]),
 'QA': array([253,  67,  38,  21])}

In [156]:
mean_total_y_num_40 = {}
std_total_y_num_40 = {}

for method in method_list_40:
    mean_total_num_list = []
    std_total_num_list = []
    for bit in bit_list_40:
        total_num_list = list(y_dic['total_y_num'][method][bit].values())
        mean_total_num_list.append(statistics.mean(total_num_list))
        std_total_num_list.append(statistics.pstdev(total_num_list))

    mean_total_y_num_40[method] = np.array(mean_total_num_list)
    std_total_y_num_40[method] = np.array(std_total_num_list)

In [157]:
mean_total_y_num_40

{'SA': array([9745, 9494, 9616, 9645]), 'QA': array([253,  67,  38,   3])}

### mean_occurrence

In [158]:
mean_occurrence_35 = {}
std_occurrence_35 = {}

for method in method_list_35:
    mean_occurrence_list = []
    std_occurrence_list = []
    for bit in bit_list_35:
        occurrence_list = list(y_dic['mean_occurrence'][method][bit].values())
        mean_occurrence_list.append(statistics.mean(occurrence_list))
        std_occurrence_list.append(statistics.pstdev(occurrence_list))

    mean_occurrence_35[method] = np.array(mean_occurrence_list)
    std_occurrence_35[method] = np.array(std_occurrence_list)

In [160]:
mean_occurrence_35

{'random': array([1.00125, 1.     , 1.     , 1.     ]),
 'SA': array([11,  2,  1,  1]),
 'QA': array([1, 1, 1, 1])}

In [161]:
mean_occurrence_40 = {}
std_occurrence_40 = {}

for method in method_list_40:
    mean_occurrence_list = []
    std_occurrence_list = []
    for bit in bit_list_40:
        occurrence_list = list(y_dic['mean_occurrence'][method][bit].values())
        mean_occurrence_list.append(statistics.mean(occurrence_list))
        std_occurrence_list.append(statistics.pstdev(occurrence_list))

    mean_occurrence_40[method] = np.array(mean_occurrence_list)
    std_occurrence_40[method] = np.array(std_occurrence_list)

In [162]:
mean_occurrence_40

{'SA': array([11,  2,  1,  1]), 'QA': array([1, 1, 1, 1])}